In [1]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

In [2]:
#Cargo los sets
X_train = pd.read_csv('data/X_train_ohc.csv', dtype={'id':'int',\
                                           'tipodepropiedad':'category',\
                                           'ciudad':'category', 'provincia':'category',\
                                           'antiguedad':'float', 'habitaciones':'float',\
                                           'garages':'float','banos':'float',\
                                           'metroscubiertos':'float', 'metrostotales':'float',\
                                           'idzona':'float',\
                                           'gimnasio':'bool', 'usosmultiples':'bool',\
                                           'piscina':'bool', 'escuelascercanas':'bool',\
                                           'centroscomercialescercanos':'bool'\
                                           }, index_col='id')
y_train = pd.read_csv('data/y_train.csv',header=None,squeeze=True,index_col=0)
X_test = pd.read_csv('data/X_test_ohc.csv', dtype={'id':'int',\
                                           'tipodepropiedad':'category',\
                                           'ciudad':'category', 'provincia':'category',\
                                           'antiguedad':'float', 'habitaciones':'float',\
                                           'garages':'float','banos':'float',\
                                           'metroscubiertos':'float', 'metrostotales':'float',\
                                           'idzona':'float',\
                                           'gimnasio':'bool', 'usosmultiples':'bool',\
                                           'piscina':'bool', 'escuelascercanas':'bool',\
                                           'centroscomercialescercanos':'bool'\
                                           }, index_col='id')
y_test = pd.read_csv('data/y_test.csv',header=None,squeeze=True,index_col=0)

In [3]:
X_submit = pd.read_csv('data/X_submit_ohc.csv', dtype={'id':'int',\
                                           'tipodepropiedad':'category',\
                                           'ciudad':'category', 'provincia':'category',\
                                           'antiguedad':'float', 'habitaciones':'float',\
                                           'garages':'float','banos':'float',\
                                           'metroscubiertos':'float', 'metrostotales':'float',\
                                           'idzona':'float',\
                                           'gimnasio':'bool', 'usosmultiples':'bool',\
                                           'piscina':'bool', 'escuelascercanas':'bool',\
                                           'centroscomercialescercanos':'bool'\
                                           }, index_col='id')

In [4]:
randomForest = RandomForestRegressor(n_estimators=1200, n_jobs= 5, oob_score=True, min_samples_leaf=2, max_features= 0.5)

In [5]:
def RMSE(actual, pred):
    return np.sqrt(mean_squared_error(actual, pred))

In [6]:
y_train_log = np.log1p(y_train)
y_test_log = np.log1p(y_test)

In [7]:
def print_score(r,train_set,test_set,train_target,test_target,F):
    res = [F(r.predict(train_set), train_target),
           F(r.predict(test_set), test_target),
           r.score(train_set, train_target), r.score(test_set, test_target)]
    if hasattr(r, 'oob_score_'): res.append(r.oob_score_)
    print(res)

In [11]:
#Dado que los precios estan acumulados en la izquierda, voy a predecir el logaritmo
#de estos, ya que se parecen a una normal, siendo los datos mas uniformes

In [ ]:
features_importancia = pd.DataFrame(randomForest.feature_importances_,\
                                      index = X_train.columns,\
                                      columns=['importancia']).sort_values('importancia', ascending=False)\
                                      

features_importancia = features_importancia[features_importancia['importancia']>0]
features_importancia.head(50)

In [ ]:
features_importancia.to_csv('data/features_importancia_primera_etapa.csv')

In [9]:
#Se puede notar una pequeña mejora en las predicciones si se predice el log

In [10]:
#Para despues hacer el submit en su momento hago np.exp(xgb_pred)

Voy a probar hacerlo con los primeros 50 features mas importantes

In [8]:
features_50 = pd.read_csv('data/features_importancia_primera_etapa.csv',index_col=0).head(60)\
            .index.tolist()
features_50

['metroscubiertos',
 'metrostotales',
 'banos',
 'provincia__Distrito Federal',
 'idzona',
 'tipodepropiedad__Apartamento',
 'garages',
 'año',
 'antiguedad',
 'dia',
 'ciudad__Huixquilucan',
 'habitaciones',
 'mes',
 'ciudad__San Pedro Garza García',
 'tipodepropiedad__Terreno',
 'tipodepropiedad__Casa',
 'ciudad__Miguel Hidalgo',
 'provincia__Edo. de México',
 'ciudad__Naucalpan de Juárez',
 'ciudad__Zapopan',
 'ciudad__Benito Juárez',
 'ciudad__Cuauhtémoc',
 'centroscomercialescercanos',
 'escuelascercanas',
 'tipodepropiedad__Terreno comercial',
 'gimnasio',
 'tipodepropiedad__Casa en condominio',
 'tipodepropiedad__Edificio',
 'piscina',
 'ciudad__San Andrés Cholula',
 'provincia__Jalisco',
 'provincia__Yucatán',
 'provincia__Nuevo León',
 'ciudad__Atizapán de Zaragoza',
 'ciudad__Alvaro Obregón',
 'usosmultiples',
 'ciudad__Mérida',
 'ciudad__Iztapalapa',
 'ciudad__Monterrey',
 'ciudad__Tlalpan',
 'ciudad__Coyoacán',
 'tipodepropiedad__Bodega comercial',
 'ciudad__Guadalajara',
 

In [9]:
X_train_50 = X_train[features_50]
X_test_50 = X_test[features_50]

In [10]:
%time randomForest.fit(X_train_50, y_train_log)

CPU times: user 30min 6s, sys: 6.77 s, total: 30min 13s
Wall time: 6min 42s


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=0.5, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=2, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1200, n_jobs=5,
           oob_score=True, random_state=None, verbose=0, warm_start=False)

In [11]:
%time print_score(randomForest,X_train_50,X_test_50,y_train_log,y_test_log,RMSE)

[0.17115320612297644, 0.33277134082575277, 0.9581655313636898, 0.8431388446909546, 0.8425968968982562]
CPU times: user 7min 44s, sys: 944 ms, total: 7min 44s
Wall time: 1min 34s


In [14]:
y_test_pred = randomForest.predict(X_test_50)

In [ ]:
serie_pred = pd.Series(data=y_test_pred, index= y_test.index, name='target' ).reset_index()

In [ ]:
serie_pred.to_csv('data/rf_test_pred.csv',index=False)

In [ ]:
#[0.17161489285239415, 0.3310967160138811, 0.9580282514296621, 0.8436874439242649, 0.8421187086795238]
#el mejor score sin features del log del precio con 1200 estimadores y 60 features